## Setup
To use this notebook, run the following to install the dependencies. Note, the first time you use anything (library, function, etc), Julia compiles it. This means that writing `using CUDA` takes ~1m42s on my workstation the first time I run it. Be warned!

In [28]:
using Pkg; Pkg.add("CUDA"); Pkg.add("IntervalOptimisation"); Pkg.add("Symbolics");

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## First adventures

Julia is like python but also like C++. It also has tons of built in math, like numpy-style array operations are in by default

In [17]:
#An example of array arithmetic
v = [1.0, 2,3]
v+v

3-element Vector{Float64}:
 2.0
 4.0
 6.0

Note that Julia determined the type (Float64) from the literal's used (in this case the 1.0). It naturally has mathematical operations defined too, like vector addition. I can even do a dot product   

In [4]:
using LinearAlgebra
dot(v,v)

14.0

This isn't sexy enough. Write `v ` then `\cdot` and press tab to autocomplete (yes latex is the way these are defined), then put `v` and you get a dot product 

In [18]:
v ⋅ v

14.0

That's right, we have unicode, and a lot of it is plumbed into real mathematical definitions. Lets make a function, one that uses Julia's cool implicit multiply

In [62]:
f(x) = 2 + 5x

f (generic function with 1 method)

What is f?

In [64]:
code_lowered(f)

1-element Vector{Core.CodeInfo}:
 CodeInfo(
1 ─ %1 = 5 * x
│   %2 = 2 + %1
└──      return %2
)

Cool! I can see the code! Hmm, looks a little generic and slow. What if I put the type information in, do I see more code?

In [67]:
code_typed(f, (Float64,))

1-element Vector{Any}:
 CodeInfo(
1 ─ %1 = Base.mul_float(5.0, x)::Float64
│   %2 = Base.add_float(2.0, %1)::Float64
└──      return %2
) => Float64

Its compiling to some specialised operations for floats where its tracking the types, the values, and the arguments. Do we hit bare metal?

In [70]:
code_native(f, (Float64,))

	.text
	.file	"f"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_f_8971
.LCPI0_0:
	.quad	0x4014000000000000              # double 5
.LCPI0_1:
	.quad	0x4000000000000000              # double 2
	.text
	.globl	julia_f_8971
	.p2align	4, 0x90
	.type	julia_f_8971,@function
julia_f_8971:                           # @julia_f_8971
; ┌ @ /home/mjki2mb2/testJulia/test.ipynb:1 within `f`
	.cfi_startproc
# %bb.0:                                # %top
	movabsq	$.LCPI0_0, %rax
; │┌ @ promotion.jl:389 within `*` @ float.jl:385
	vmulsd	(%rax), %xmm0, %xmm0
	movabsq	$.LCPI0_1, %rax
; │└
; │┌ @ promotion.jl:388 within `+` @ float.jl:383
	vaddsd	(%rax), %xmm0, %xmm0
; │└
	retq
.Lfunc_end0:
	.size	julia_f_8971, .Lfunc_end0-julia_f_8971
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


Awesome, assembly! All created whenever we use the function, but not before! So we always get compiled, fully optimised code. Wait, if we write abstract functions, that then get adapted to native code only when run, can we run on non-native devices?
# CUDA
Yes, just change the type so the compiler can infer we run it elsewhere.

In [12]:
using CUDA
gpu_v = CuArray([1,2,3,4,5,6,7,8]) #This array now lives on the GPU

#Some generic function using the . to make .+ an elementwise addition
f(x) = x .+ 12 

f(gpu_v)

8-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 13
 14
 15
 16
 17
 18
 19
 20

I gotta see what this looks like in GPU assembly (warning, its filled with inlined code from the library)

In [25]:
using InteractiveUtils #We use the macro @code_native from here as its a bit easier to specify the function specialisation by a variable/call
@code_native f(gpu_v)

	.text
	.file	"f"
	.section	.rodata.cst16,"aM",@progbits,16
	.p2align	4                               # -- Begin function julia_f_5246
.LCPI0_0:
	.quad	1                               # 0x1
	.quad	12                              # 0xc
	.text
	.globl	julia_f_5246
	.p2align	4, 0x90
	.type	julia_f_5246,@function
julia_f_5246:                           # @julia_f_5246
; ┌ @ /home/mjki2mb2/testJulia/test.ipynb:5 within `f`
	.cfi_startproc
# %bb.0:                                # %top
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$448, %rsp                      # imm = 0x1C0
	.cfi_offset %rbx, -56
	.cfi_offset %r12, -48
	.cfi_offset %r13, -40
	.cfi_offset %r14, -32
	.cfi_offset %r15, -24
	vxorpd	%xmm0, %xmm0, %xmm0
	vmovapd	%ymm0, 160(%rsp)
	vmovapd	%ymm0, 128(%rsp)
	movabsq	$140561802599280, %r15          # imm = 0x7FD718501B70
	vmovapd	%ymm0, 96(%rsp


; │││││┌ @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:19 within `ArrayStorage`
; ││││││┌ @ atomics.jl:78 within `Atomic`
	movq	16(%r13), %rdi
	movabsq	$ijl_gc_pool_alloc, %r12
	movl	$1392, %esi                     # imm = 0x570
	movl	$16, %edx
	callq	*%r12
	movq	%rax, %rbx
	leaq	712144(%r14), %rax
	movq	%rax, -8(%rbx)
	movq	$1, (%rbx)
	movq	%rbx, 144(%rsp)
; │││││└└
; │││││ @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:136 within `CuArray` @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:125 @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:44
	movq	16(%r13), %rdi
	movl	$1488, %esi                     # imm = 0x5D0
	movl	$48, %edx
	callq	*%r12
	movq	%rax, %r14
	leaq	-13916224(%r15), %rax
	movq	%rax, -8(%r14)
	movq	$0, (%r14)
	movq	%r14, 184(%rsp)
	movq	16(%r13), %rdi
	movl	$1488, %esi                     # imm = 0x5D0
	movl	$48, %edx
	callq	*%r12
	leaq	-13915904(%r15), %rcx
	movq	%rcx, -8(%rax)
	movq	64(%rsp), %rcx                  # 8-byte Reload

$9223372036854775807, %r15      # imm = 0x7FFFFFFFFFFFFFFF
; │││││└
	movq	%rbx, 368(%rsp)
	setne	376(%rsp)
; │││││┌ @ boot.jl:603 within `NamedTuple` @ boot.jl:607
	movq	%r15, 208(%rsp)
	movq	%rbx, 128(%rsp)
	movabsq	$.LCPI0_0, %rax
; │││││└
	vmovaps	(%rax), %xmm0
	vmovups	%xmm0, 384(%rsp)
	movq	56(%rsp), %rax                  # 8-byte Reload
	movq	%rax, 400(%rsp)
	movq	%rbx, 40(%rsp)                  # 8-byte Spill
	movq	%rbx, 176(%rsp)
	movabsq	$"j_launch_heuristic##kw_5249", %rax
	leaq	408(%rsp), %r12
	leaq	200(%rsp), %rsi
	leaq	368(%rsp), %rcx
	movl	$1, %r8d
	movq	%r12, %rdi
	movq	%r14, %rdx
	callq	*%rax
; │││││ @ /home/mjki2mb2/.julia/packages/GPUArrays/fqD8z/src/host/broadcast.jl:65 within `_copyto!`
; │││││┌ @ /home/mjki2mb2/.julia/packages/GPUArrays/fqD8z/src/device/execution.jl:88 within `launch_configuration##kw`
	movabsq	$"j_#launch_configuration#3_5250", %rax
	leaq	216(%rsp), %rdi
	movq	%r13, %rsi
	movq	%r15, %rdx
	movq	%r12, %rcx
	callq	*%rax
; │││││└
; │││││ @ /home/mjki2

%rdi
	callq	*%rbx
	movq	%rax, %r13
	movq	%rax, 160(%rsp)
	movq	16(%rsp), %rbx                  # 8-byte Reload
	movq	16(%rbx), %rdi
	movl	$1440, %esi                     # imm = 0x5A0
	movl	$32, %edx
	movabsq	$ijl_gc_pool_alloc, %r15
	callq	*%r15
	movq	%r15, %rcx
	movq	%rax, %r15
	movabsq	$140561802599280, %r12          # imm = 0x7FD718501B70
	leaq	-446699264(%r12), %rax
	movq	%rax, -8(%r15)
	movq	360(%rsp), %rax
	movq	%rax, 16(%r15)
	vmovupd	344(%rsp), %xmm0
	vmovupd	%xmm0, (%r15)
	movq	%r15, 152(%rsp)
	movq	16(%rbx), %rdi
	movl	$1488, %esi                     # imm = 0x5D0
	movl	$48, %edx
	callq	*%rcx
	movq	%rax, %rbx
	addq	$-72045776, %r12                # imm = 0xFBB4AB30
	movq	%r12, -8(%rax)
	movl	$1, %eax
	movq	%rax, 16(%rbx)
	movb	48(%rsp), %al                   # 1-byte Reload
	movb	%al, 8(%rbx)
	movq	40(%rsp), %rax                  # 8-byte Reload
	movq	%rax, (%rbx)
	movq	$12, 24(%rbx)
	movq	56(%rsp), %rax                  # 8-byte Reload
	movq	%rax, 32(%rbx)
	movq	%rbx, 144(%


; │││││││││││││││┌ @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:276 within `pointer`
; ││││││││││││││││┌ @ /home/mjki2mb2/.julia/packages/CUDA/DfvRa/src/array.jl:321 within `unsafe_convert`
	movabsq	$j_getproperty_5251, %rax
	movabsq	$140566802556928, %rdi          # imm = 0x7FD842556800
	callq	*%rax
	ud2
.LBB0_22:                               # %L121
	movabsq	$ijl_throw, %rax
	movabsq	$140566773045744, %rdi          # imm = 0x7FD8409319F0
	callq	*%rax
.LBB0_23:                               # %L133
; ││││││││││││││└└└
; ││││││││││││││ @ /home/mjki2mb2/.julia/packages/Adapt/LAQOx/src/base.jl:30 within `adapt_structure`
; ││││││││││││││┌ @ /home/mjki2mb2/.julia/packages/Adapt/LAQOx/src/Adapt.jl:40 within `adapt`
; │││││││││││││││┌ @ /home/mjki2mb2/.julia/packages/Adapt/LAQOx/src/base.jl:3 within `adapt_structure`
; ││││││││││││││││┌ @ tuple.jl:222 within `map`
; │││││││││││││││││┌ @ operators.jl:1096 within `Fix1`
; ││││││││││││││││││┌ @ /home/mjki2mb2/.julia/packages/Adap

# Symbolics

In [31]:
using Symbolics

@variables x, y, t

x^2 + y^2

x^2 + y^2

How about discovering the governing differential equations behind data https://github.com/SciML/DataDrivenDiffEq.jl?
Global non-linear constrained optimisation (for small enough dimensionality) https://github.com/PSORLab/EAGO.jl